# USO DI LIBRERIE PER IMPLEMENTARE ZNE SU FAKE BACKEND

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as la
import math


from qiskit import QuantumCircuit, transpile
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.visualization import plot_histogram
from qiskit.quantum_info import SparsePauliOp, Statevector
from qiskit.primitives import StatevectorEstimator, BackendEstimatorV2 as BackendEstimator, BackendSamplerV2 as BackendSampler
from qiskit_ibm_runtime import QiskitRuntimeService
 


from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit_aer.primitives import EstimatorV2 as Estimator
from qiskit_aer.noise import (QuantumError, depolarizing_error, amplitude_damping_error, phase_damping_error, pauli_error)
from qiskit.providers import Options

#Rendiamo generale la funzione, prende N numero di Spin, t il tempo, j il primo spin e i il secondo spin (questi saranno j e j+1 in quanto nearest neighbours)
def R_zz(N, t, j, i):
    
    ZZ_qc = QuantumCircuit(N)

    ZZ_qc.cx(j,i)
    ZZ_qc.rz(2 * t, i) #vedi definizione sopracitata per il fattore 2
    ZZ_qc.cx(j,i)

    return ZZ_qc

#decomponiamo il Rot. Gate XX
def R_xx(N, t, j, i):

    XX_qc = QuantumCircuit(N)

    XX_qc.ry(np.pi/2,[j,i])
    XX_qc.cx(j,i)
    XX_qc.rz(2 * t, i)
    XX_qc.cx(j,i)
    XX_qc.ry(-np.pi/2,[j,i])

    return XX_qc

#decomponiamo il Rot. Gate YY
def R_yy(N, t, j, i):
    YY_qc = QuantumCircuit(N)
    
    YY_qc.rx(np.pi/2,[j,i])
    YY_qc.cx(j,i)
    YY_qc.rz(2 * t, i)
    YY_qc.cx(j,i)
    YY_qc.rx(-np.pi/2,[j,i])

    return YY_qc


#definiamo l'hamiltoniana
def H_Heis(N):
    #usiamo J = 1
    XX_tuples = [("XX", [i, i + 1], 1) for i in range(0, N-1)]
    YY_tuples = [("YY", [i, i + 1], 1) for i in range(0, N-1)]
    ZZ_tuples = [("ZZ", [i, i + 1], 1) for i in range(0, N-1)]

    #col metodo from_sparse_list di SparsePauliOp definiamo l'hamiltoniana
    hamiltonian = SparsePauliOp.from_sparse_list([*XX_tuples, *YY_tuples, *ZZ_tuples], N)

    return hamiltonian

#definiamo l'evoluzione temporale nel caso classico
def U_Heis(n_spins,t):
    H = H_Heis(n_spins).to_matrix()
    return la.expm(-1j*t*H)

def Heisenberg_Trotter(num_spins, trotter_steps, t):
    #definiamo un time step dt
    dt = (t)/(trotter_steps) 

    #creiamo il quantumcircuit
    qc_t = QuantumCircuit(num_spins)

    #iteriamo su ogni time step
    for i in range(trotter_steps):
        #iteriamo su ogni spin
        for j in range(0, num_spins -1):
            #implementazione dei gate è uguale al caso precedente, in quanto operiamo su coppie di spin
            qc_t = qc_t.compose(R_xx(num_spins,dt,j,j+1))
            qc_t = qc_t.compose(R_yy(num_spins,dt,j,j+1))
            qc_t = qc_t.compose(R_zz(num_spins,dt,j,j+1))

        
        #qc.barrier()

    return qc_t


#definiamo il numero di time steps
t_steps = 63

#definiamo il tempo in cui facciamo evolvere il sistema
dt = np.pi

#generiamo una spaziatura lineare di t_steps nell'intervallo [0, pi]
ts = np.linspace(0, dt, t_steps) 

initial_state = Statevector.from_label('011').data
probs_110 = [np.abs((initial_state @ U_Heis(3,float(t)) @ initial_state))**2 for t in ts]

In [2]:
#pip install mitiq[qiskit]
from mitiq import zne
from mitiq import rem
from mitiq.zne import execute_with_zne
from mitiq.zne.scaling import (
    fold_gates_at_random,
    fold_global,
    fold_all
)
from mitiq.zne.inference import LinearFactory, RichardsonFactory
from mitiq import (
    Calibrator,
    Settings,
    execute_with_mitigation,
    MeasurementResult,
)


from qiskit_ibm_runtime.fake_provider import FakeManilaV2
QiskitRuntimeService.save_account(channel="ibm_quantum", token="982082508a1c98ca02cb712c6751b42ef860d905b080b3bfc02ef35d68d84441803586912cbcf181e1e72ebc8569103421b06c25408fe8134975a72575c4d637", overwrite= True)


def execute_circuit(circuit):
    
    prj_zero = SparsePauliOp.from_list([("I", 1/2), ("Z", 1/2)])
    prj_one = SparsePauliOp.from_list([("I", 1/2), ("Z", -1/2)])
    prj = prj_one ^ prj_one ^ prj_zero
    
    isa_circuit = pm.run(circuit)
    isa_observable = prj.apply_layout(isa_circuit.layout)
    print("isa applied")
    trotter_result = estimator.run([(isa_circuit, isa_observable)]).result()
    print("circuito runnato")
    trotter_value = trotter_result[0].data.evs
    trotter_expectation_value = trotter_value.mean().real
    print(str(trotter_expectation_value) + "\n")
    return trotter_expectation_value


In [3]:
fake_backend = FakeManilaV2()
noise_model = NoiseModel.from_backend(fake_backend)
simulator = AerSimulator(noise_model=noise_model)
estimator = BackendEstimator(backend=simulator)
pm = generate_preset_pass_manager(optimization_level=0)

    
prj_zero = SparsePauliOp.from_list([("I", 1/2), ("Z", 1/2)])
prj_one = SparsePauliOp.from_list([("I", 1/2), ("Z", -1/2)])
prj = prj_one ^ prj_one ^ prj_zero

probs_110_zne = []
probs_110_trotter = []

for sim_t in ts:
    qc = QuantumCircuit(3)
    #buildiamo lo stato
    qc.x(1)
    qc.x(2)
        
    trott_steps = Heisenberg_Trotter(num_spins=3,trotter_steps=12,t=sim_t)
        
    qc = qc.compose(trott_steps)


    trotter_expectation_value = execute_with_zne(qc, execute_circuit, factory=LinearFactory([1.0, 1.5, 2.0, 2.5, 3.0]), scale_noise=zne.scaling.fold_global)
    trotter = execute_circuit(qc)


    probs_110_trotter.append(trotter)
    probs_110_zne.append(trotter_expectation_value)

    
    print(str(sim_t) + " completed\n")


isa applied
circuito runnato
0.202392578125

isa applied
circuito runnato
0.14599609375

isa applied
circuito runnato
0.124267578125

isa applied
circuito runnato
0.11572265625

isa applied
circuito runnato
0.103759765625

isa applied
circuito runnato
0.203125

0.0 completed

isa applied
circuito runnato
0.2021484375

isa applied
circuito runnato
0.158203125

isa applied
circuito runnato
0.126953125

isa applied
circuito runnato
0.1123046875

isa applied
circuito runnato
0.10791015625

isa applied
circuito runnato
0.204833984375

0.050670849251448276 completed

isa applied
circuito runnato
0.199951171875

isa applied
circuito runnato
0.148681640625

isa applied
circuito runnato
0.130126953125

isa applied
circuito runnato
0.114990234375

isa applied
circuito runnato
0.111328125

isa applied
circuito runnato
0.208251953125

0.10134169850289655 completed

isa applied
circuito runnato
0.20166015625

isa applied
circuito runnato
0.1474609375

isa applied
circuito runnato
0.120361328125

is

In [1]:
#plottiamo
plt.plot(ts, probs_110,linestyle="dashed",color="black",label="Esatta")
plt.plot(ts, probs_110_zne,label= "ZNE")
plt.plot(ts, probs_110_trotter,label= "Trotter")
plt.xlabel('t')
plt.ylabel(r'probabilità dello stato $|110\rangle$')
plt.title(r'Confronto con ZNE applicata tramite libreria mitiq')
plt.legend()
plt.grid()
plt.savefig('../../png/zne_mitiq.png')
plt.show()

NameError: name 'plt' is not defined